# A tool to prepare assembly file to run PGAP and to create input files - submol.yaml and input.yaml

In [2]:
import os

In [3]:
#inputs
assembly_file = "/home/dzilov/scripts/pgap_preparer/test_data/scaffolds.fasta"
adapters_file = "/home/dzilov/scripts/pgap_preparer/adapters_db/adaptor_fasta.fna"

#outputs
blast_out = "/home/dzilov/scripts/pgap_preparer/results/blast_results.outfmt6"
out_assembly = "/home/dzilov/scripts/pgap_preparer/results/scaffolds_pp.fasta"
submol_yaml =  "/home/dzilov/scripts/pgap_preparer/results/submol.yaml"
input_yaml =  "/home/dzilov/scripts/pgap_preparer/results/input.yaml"

In [4]:
command = """
blastn -query %s \
                    -db %s \
                    -outfmt 6 \
                    -max_target_seqs 1000 \
                    -task blastn \
                    -reward 1 \
                    -penalty \
                    -5 \
                    -gapopen 3 \
                    -gapextend 3 \
                    -dust yes \
                    -soft_masking true \
                    -evalue 0.00001 \
                    -searchsp 1750000000000 \
                    -out %s
          """ % (assembly_file, adapters_file, blast_out)

print(command)
os.system(command)


blastn -query /home/dzilov/scripts/pgap_preparer/test_data/scaffolds.fasta                     -db /home/dzilov/scripts/pgap_preparer/adapters_db/adaptor_fasta.fna                     -outfmt 6                     -max_target_seqs 1000                     -task blastn                     -reward 1                     -penalty                     -5                     -gapopen 3                     -gapextend 3                     -dust yes                     -soft_masking true                     -evalue 0.00001                     -searchsp 1750000000000                     -out /home/dzilov/scripts/pgap_preparer/results/blast_results.outfmt6
          


0

In [5]:
seq_to_del = {}
with open(blast_out) as fh:
    for line in fh:
        line = line.strip().split()
        ids = line[1].split(":")[1]
        contig = line[0]
        start = line[6]
        stop = line[7]
        seq_to_del[contig] = start + ":" + stop
        print(ids, contig, start, stop)
print(seq_to_del) 

NGB00736.1 NODE_61_length_8737_cov_90.704181 8552 8597
NGB00728.1 NODE_148_length_482_cov_0.949296 321 350
{'NODE_61_length_8737_cov_90.704181': '8552:8597', 'NODE_148_length_482_cov_0.949296': '321:350'}


In [6]:
fasta = {}
header = None
with open(assembly_file) as fh:
    for i, line in enumerate(fh):
        line = line.strip()
        if line.startswith(">"):
            if header:
                fasta[header]="".join(seq)
            header = line
            seq = []
        else:
            seq.append(line)
if header:
    fasta[header] = "".join(seq)

In [7]:
for contig, position in seq_to_del.items():
    start = int(position.split(":")[0])-1
    stop = int(position.split(":")[1])
    for key, value in fasta.items():
        if key.startswith(">" + contig):
            line_to_del = value[start:stop]
            value = value.replace(line_to_del, "")
            fasta[key] = value

In [8]:
short_contig = []
for key, value in fasta.items():
    if len(value) < 200:
        short_contig.append(key)

for i in short_contig:
    if i in fasta.keys():
        fasta.pop(i)

0


In [27]:
with open(out_assembly, "w") as fw:
    for key, value in fasta.items():
        fw.write(key + "\n" + value + "\n")
fw.close()

In [42]:
bact = "Escherichia coli"
last_name = "Zilov"
first_name = "Danil"
email = "zilov@scamt-itmo.ru"
organization = "SCAMT Institute, ITMO University"
department = "Applied Genomics Laboratory"
street = "9, Lomonosova street"
city = "Saint-Petersburg"
state = "Saint-Petersburg"
postal_code = "191002"
country = "Russian Federation"
author_last_name = 'Zilov'        
author_first_name = 'Danil'

In [47]:
with open(submol_yaml, 'w') as fw:
    fw.write("""comment: 'There is no really a biologist {first_name} {last_name}'
consortium: 'SkyNet consortium'
organism:
    genus_species: '{bact}' 
    strain: 'replaceme'
contact_info:
    last_name: '{last_name}'
    first_name: '{first_name}'
    email: '{email}'
    organization: '{organization}'
    department: '{department}'
    phone: '301-555-0245'
    street: '{street}'
    city: '{city}'
    postal_code: '{postal_code}'
    country: '{country}'
    
authors:
    -     author:
            first_name: '{author_first_name}'
            last_name: '{author_last_name}'""" .format(bact=bact, last_name=last_name, first_name=first_name,
                        email=email, organization=organization, department=department,
                        street=street, city=city,postal_code=postal_code,country=country,
                        author_last_name=author_last_name, author_first_name=author_first_name))
fw.close()

In [50]:
with open(input_yaml, 'w') as fw:
    fw.write("""fasta:
    class: File
    location: {assembly}
submol:
  class: File
  location: {submol_yaml}
    """.format(assembly=out_assembly, submol_yaml=submol_yaml))
fw.close()